In [54]:
# from sentence_transformers import SentenceTransformer
import pandas as pd
# import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import numpy as np
import json
import joblib
import requests

from sacrebleu.metrics import BLEU
from rouge import Rouge
import statistics

In [55]:
state = 'NY'
kmeans = joblib.load(f"{state}_kmeans.pkl")
scaler = joblib.load(f"{state}_scaler.pkl")
weights = [5,5,1,1,1,1,1,1,1]
cols = ['bedrooms', 'bathrooms', 'price', 'livingArea', 'leisure_within_5km', 'shops_within_5km', 'schools_within_5km', 'transit_within_2km', 'homeType']

In [56]:
predict_X=[2,1,1700000,2000,20,400,250,200,2]  # give input here
description = "A large backyard, swimming pool, spacious living room, furniture in bedroom, tiled bathroom"

In [57]:
json_file_path = "./ClusterPoints_Dataset.json"

with open(json_file_path, "r") as file:
    kmeansdata = json.load(file)

json_file_path_1 = "./Labelled_Dataset_with_zpid.json"

with open(json_file_path_1, "r") as file:
    fulldata = json.load(file)

In [58]:
df_predict = pd.DataFrame([predict_X],columns=cols)
scaler_predict=StandardScaler()

df_to_transform_p = df_predict.drop(columns=['homeType'])
df_excluded_p = df_predict[['homeType']]
# Apply the transformation
transformed_array_p = scaler.transform(df_to_transform_p)
print(transformed_array_p)
df_transformed_p = pd.DataFrame(transformed_array_p, columns=df_to_transform_p.columns, index=df_predict.index)
# Recombine the DataFrame
df_result_p = pd.concat([df_transformed_p, df_excluded_p], axis=1)
# print(df_result)
X_p=df_result_p.values
X_p=X_p*weights
X_p

[[ 0.09090596 -0.73108393 -0.20746602  0.8198349  -0.35234394 -0.98294832
   2.22322311 -0.88709456]]


array([[ 0.45452982, -3.65541963, -0.20746602,  0.8198349 , -0.35234394,
        -0.98294832,  2.22322311, -0.88709456,  2.        ]])

In [59]:
predicted_label=kmeans.predict(X_p)
predicted_label

array([11], dtype=int32)

In [60]:
state_full_data=fulldata[state]
state_full_data

[{'zpid': 31491229,
  'bedrooms': 1,
  'bathrooms': 1,
  'price': 650000.0,
  'homeType': 1,
  'livingArea': 776.0,
  'leisure_within_5km': 21,
  'shops_within_5km': 540,
  'schools_within_5km': 128,
  'transit_within_2km': 121,
  'labels': 1},
 {'zpid': 31492996,
  'bedrooms': 2,
  'bathrooms': 2,
  'price': 959000.0,
  'homeType': 1,
  'livingArea': 1000.0,
  'leisure_within_5km': 19,
  'shops_within_5km': 538,
  'schools_within_5km': 127,
  'transit_within_2km': 110,
  'labels': 3},
 {'zpid': 31493494,
  'bedrooms': 1,
  'bathrooms': 1,
  'price': 1100000.0,
  'homeType': 1,
  'livingArea': 815.0,
  'leisure_within_5km': 20,
  'shops_within_5km': 614,
  'schools_within_5km': 141,
  'transit_within_2km': 194,
  'labels': 1},
 {'zpid': 31494186,
  'bedrooms': 0,
  'bathrooms': 1,
  'price': 750000.0,
  'homeType': 1,
  'livingArea': 465.0,
  'leisure_within_5km': 22,
  'shops_within_5km': 717,
  'schools_within_5km': 160,
  'transit_within_2km': 272,
  'labels': 5},
 {'zpid': 31494718

In [61]:
def filter_by_labels(data, labels):
    # Use list comprehension to get dictionaries with matching labels
    filtered_data = [entry for entry in data if entry['labels'] == labels]
    return filtered_data

In [62]:
state_filtered_data = filter_by_labels(state_full_data, predicted_label[0])
len(state_filtered_data)

27

In [63]:
state_filtered_data

[{'zpid': 31495264,
  'bedrooms': 2,
  'bathrooms': 1,
  'price': 798000.0,
  'homeType': 1,
  'livingArea': 525.0,
  'leisure_within_5km': 28,
  'shops_within_5km': 836,
  'schools_within_5km': 181,
  'transit_within_2km': 268,
  'labels': 11},
 {'zpid': 31507674,
  'bedrooms': 2,
  'bathrooms': 1,
  'price': 1350000.0,
  'homeType': 1,
  'livingArea': 1053.0,
  'leisure_within_5km': 21,
  'shops_within_5km': 762,
  'schools_within_5km': 182,
  'transit_within_2km': 346,
  'labels': 11},
 {'zpid': 31527126,
  'bedrooms': 2,
  'bathrooms': 1,
  'price': 875000.0,
  'homeType': 1,
  'livingArea': 0.0,
  'leisure_within_5km': 21,
  'shops_within_5km': 712,
  'schools_within_5km': 170,
  'transit_within_2km': 403,
  'labels': 11},
 {'zpid': 31545643,
  'bedrooms': 2,
  'bathrooms': 1,
  'price': 1050000.0,
  'homeType': 1,
  'livingArea': 0.0,
  'leisure_within_5km': 23,
  'shops_within_5km': 683,
  'schools_within_5km': 176,
  'transit_within_2km': 294,
  'labels': 11},
 {'zpid': 3154675

In [64]:
df = pd.DataFrame(state_filtered_data)
df=df.set_index(keys=['zpid'])
df

,bedrooms,bathrooms,price,homeType,livingArea,leisure_within_5km,shops_within_5km,schools_within_5km,transit_within_2km,labels
zpid,,,,,,,,,,
31495264,2,1,798000.0,1,525.000000,28,836,181,268,11
31507674,2,1,1350000.0,1,1053.000000,21,762,182,346,11
31527126,2,1,875000.0,1,0.000000,21,712,170,403,11
31545643,2,1,1050000.0,1,0.000000,23,683,176,294,11
31546758,2,1,649000.0,1,755.000000,29,492,212,246,11
61968154,2,1,1590000.0,1,884.000000,24,659,186,300,11
72532295,2,1,799000.0,1,0.000000,25,543,209,272,11
72534234,3,1,885000.0,1,1212.000000,28,402,205,345,11
72535178,2,1,1049000.0,1,0.000000,24,350,185,216,11


In [65]:
zpid_list = []
for i in range(len(state_filtered_data)):
    zpid_list.append(state_filtered_data[i]['zpid'])

with open("./data_without_ic.json", "r") as f:  ## change embeddings file here
    data_embed = json.load(f)
# new_data_point = file_embeddings[str(zpid_list[0])]
print(zpid_list)
dict_embeddings = {}
for i in zpid_list:
    if str(i) in data_embed.keys():
        dict_embeddings[i] = data_embed[str(i)]

[31495264, 31507674, 31527126, 31545643, 31546758, 61968154, 72532295, 72534234, 72535178, 82784715, 89330466, 97537243, 119911138, 119987636, 121999838, 244754141, 244793565, 244876319, 244898624, 342149156, 347350097, 347521090, 347756370, 2058409968, 2112868113, 2121084357, 2139218516]


In [66]:
payload = {
    'embeddings': dict_embeddings,
    'message': description
}

# Define the URL of the backend server
server_url = "http://127.0.0.1:5000/predict"  # Replace with your server URL

# Send a POST request
response = requests.post(
    server_url,
    json=payload,  # Sending data as JSON
    headers={'Content-Type': 'application/json'}  # Specifying content type
)

# Check if the request was successful
if response.status_code == 200:
    print("Data sent successfully!")
else:
    print("Failed to send data. Status code:", response.status_code)

Data sent successfully!


In [67]:
zpid_sim = response.json()['similarities']
    # print(type(zpid_sim))
json_file_path = "./combined_summary_data.json"

zpid_list = [str(zpid) for zpid, _ in zpid_sim]
with open(json_file_path, "r",encoding="utf-8", errors='ignore') as file:
    data = json.load(file)

# filtered_houses = [house for house in data if str(house["zpid"]) in zpid_list]
filtered_houses = [data[zpid] for zpid in zpid_list if zpid in data]

# Recommendation logic using the selected state
recommended_properties = filtered_houses
recommendations = pd.DataFrame(recommended_properties, index=zpid_list)

In [68]:
recommendations

,bedrooms,bathrooms,price,zipcode,hdpUrl,cityId,homeType,livingArea,livingAreaValue,rentZestimate,...,address.city,address.state,address.zipcode,image_captions,leisure_within_5km,shops_within_5km,schools_within_5km,transit_within_2km,concatenated_desc,summary
61968154,2.0,1.0,1590000,10075,/homedetails/215-E-80th-St-APT-9A-New-York-NY-...,6181.0,CONDO,884.0,884.0,4500.0,...,New York,NY,10075,a kitchen with a bar and a counter with stools...,24,659,186,300,For the discerning buyer who deserves a brand ...,"A luxury, sophisticated, diamond in the rough...."
72532295,2.0,1.0,799000,10128,/homedetails/226-E-95th-St-APT-302-New-York-NY...,6181.0,CONDO,0.0,0.0,4206.0,...,New York,NY,10128,"a living room with a couch, chair, and televis...",25,543,209,272,Two Bedrooms for the price of One!\nHere is yo...,Two bedrooms for the price of one. The apartme...
31527126,2.0,1.0,875000,10019,/homedetails/77-W-55th-St-APT-3E-New-York-NY-1...,6181.0,CONDO,0.0,0.0,4300.0,...,New York,NY,10019,"a living room with a couch, coffee table, and ...",21,712,170,403,Experience the pinnacle of Midtown living in t...,26-foot living room bathes in natural light fr...
2112868113,2.0,1.0,850000,10065,/homedetails/1160-3rd-Ave-APT-10D-New-York-NY-...,6181.0,CONDO,0.0,0.0,5183.0,...,New York,NY,10065,"a living room with a couch, coffee table, and ...",21,744,182,352,"Bespoke two-bedroom, one bath residence with s...","Bespoke two-bedroom, one bath residence with s..."
72535178,2.0,1.0,1049000,10025,/homedetails/417-Riverside-Dr-APT-2-D-New-York...,6181.0,CONDO,0.0,0.0,3864.0,...,New York,NY,10025,"a living room with a couch, coffee table, and ...",24,350,185,216,"GRACIOUS TWO-BEDROOM\nGracious, renovated and ...","10ft ceilings, French doors, in-laid oak floor..."
244876319,2.0,1.0,675000,10017,/homedetails/2-Tudor-City-Pl-APT-3DS-New-York-...,6181.0,CONDO,900.0,900.0,3999.0,...,New York,NY,10017,"a living room with a couch, chair, table and a...",21,792,193,325,Amazing value in the heart of Tudor City. An o...,2 Tudor City Place is a full-service 24-hour d...
119911138,2.0,1.0,529000,10034,/homedetails/98-Park-Ter-E-APT-1C-New-York-NY-...,6181.0,CONDO,1000.0,1000.0,2653.0,...,New York,NY,10034,a living room with a blue couch and a televisi...,37,80,115,103,Vintage Charm with a Modern Twist: Spacious 2B...,"Two-bedroom home in the heart of Inwood, New Y..."
342149156,2.0,1.0,1250000,10023,/homedetails/30-W-60th-St-8C-New-York-NY-10023...,6181.0,CONDO,1100.0,1100.0,5550.0,...,New York,NY,10023,"a living room with a couch, chair, and a table...",21,619,161,323,Come check out this very rare unit! This line ...,This amazing 1100sqft 2 bed 1 bath has not com...
347756370,2.0,1.0,1100000,10025,/homedetails/215-W-92nd-St-APT-2G-New-York-NY-...,6181.0,CONDO,1174.0,1174.0,NaN,...,New York,NY,10025,a living room with a piano and a piano table i...,19,438,169,216,Step into timeless elegance with this charming...,"2 bedroom, 1 bathroom residence nestled in The..."
89330466,2.0,1.0,1550000,10069,/homedetails/100-Riverside-Blvd-APT-28E-New-Yo...,6181.0,CONDO,1000.0,1000.0,6501.0,...,New York,NY,10069,a tall building with many windows in a city ar...,19,523,152,201,100 Riveside Blvd Apartment 28 E\nSpacious and...,"Spacious and Sunny 1,000 SF Two Bedroom Corner..."


In [69]:
top_10_recommendations = recommendations.head(10)
top_10_recommendations

,bedrooms,bathrooms,price,zipcode,hdpUrl,cityId,homeType,livingArea,livingAreaValue,rentZestimate,...,address.city,address.state,address.zipcode,image_captions,leisure_within_5km,shops_within_5km,schools_within_5km,transit_within_2km,concatenated_desc,summary
61968154,2.0,1.0,1590000,10075,/homedetails/215-E-80th-St-APT-9A-New-York-NY-...,6181.0,CONDO,884.0,884.0,4500.0,...,New York,NY,10075,a kitchen with a bar and a counter with stools...,24,659,186,300,For the discerning buyer who deserves a brand ...,"A luxury, sophisticated, diamond in the rough...."
72532295,2.0,1.0,799000,10128,/homedetails/226-E-95th-St-APT-302-New-York-NY...,6181.0,CONDO,0.0,0.0,4206.0,...,New York,NY,10128,"a living room with a couch, chair, and televis...",25,543,209,272,Two Bedrooms for the price of One!\nHere is yo...,Two bedrooms for the price of one. The apartme...
31527126,2.0,1.0,875000,10019,/homedetails/77-W-55th-St-APT-3E-New-York-NY-1...,6181.0,CONDO,0.0,0.0,4300.0,...,New York,NY,10019,"a living room with a couch, coffee table, and ...",21,712,170,403,Experience the pinnacle of Midtown living in t...,26-foot living room bathes in natural light fr...
2112868113,2.0,1.0,850000,10065,/homedetails/1160-3rd-Ave-APT-10D-New-York-NY-...,6181.0,CONDO,0.0,0.0,5183.0,...,New York,NY,10065,"a living room with a couch, coffee table, and ...",21,744,182,352,"Bespoke two-bedroom, one bath residence with s...","Bespoke two-bedroom, one bath residence with s..."
72535178,2.0,1.0,1049000,10025,/homedetails/417-Riverside-Dr-APT-2-D-New-York...,6181.0,CONDO,0.0,0.0,3864.0,...,New York,NY,10025,"a living room with a couch, coffee table, and ...",24,350,185,216,"GRACIOUS TWO-BEDROOM\nGracious, renovated and ...","10ft ceilings, French doors, in-laid oak floor..."
244876319,2.0,1.0,675000,10017,/homedetails/2-Tudor-City-Pl-APT-3DS-New-York-...,6181.0,CONDO,900.0,900.0,3999.0,...,New York,NY,10017,"a living room with a couch, chair, table and a...",21,792,193,325,Amazing value in the heart of Tudor City. An o...,2 Tudor City Place is a full-service 24-hour d...
119911138,2.0,1.0,529000,10034,/homedetails/98-Park-Ter-E-APT-1C-New-York-NY-...,6181.0,CONDO,1000.0,1000.0,2653.0,...,New York,NY,10034,a living room with a blue couch and a televisi...,37,80,115,103,Vintage Charm with a Modern Twist: Spacious 2B...,"Two-bedroom home in the heart of Inwood, New Y..."
342149156,2.0,1.0,1250000,10023,/homedetails/30-W-60th-St-8C-New-York-NY-10023...,6181.0,CONDO,1100.0,1100.0,5550.0,...,New York,NY,10023,"a living room with a couch, chair, and a table...",21,619,161,323,Come check out this very rare unit! This line ...,This amazing 1100sqft 2 bed 1 bath has not com...
347756370,2.0,1.0,1100000,10025,/homedetails/215-W-92nd-St-APT-2G-New-York-NY-...,6181.0,CONDO,1174.0,1174.0,NaN,...,New York,NY,10025,a living room with a piano and a piano table i...,19,438,169,216,Step into timeless elegance with this charming...,"2 bedroom, 1 bathroom residence nestled in The..."
89330466,2.0,1.0,1550000,10069,/homedetails/100-Riverside-Blvd-APT-28E-New-Yo...,6181.0,CONDO,1000.0,1000.0,6501.0,...,New York,NY,10069,a tall building with many windows in a city ar...,19,523,152,201,100 Riveside Blvd Apartment 28 E\nSpacious and...,"Spacious and Sunny 1,000 SF Two Bedroom Corner..."


In [70]:
bleu_scorer = BLEU()

column = 'description' ## Change the column name as needed,, make it 'summary' for the best model

bleu_scores = [bleu_scorer.sentence_score(
    hypothesis=description,
    references=[reference],
).score for reference in top_10_recommendations[column].tolist()]

bleu_scores

It is recommended to enable `effective_order` for sentence-level BLEU.
It is recommended to enable `effective_order` for sentence-level BLEU.
It is recommended to enable `effective_order` for sentence-level BLEU.
It is recommended to enable `effective_order` for sentence-level BLEU.
It is recommended to enable `effective_order` for sentence-level BLEU.
It is recommended to enable `effective_order` for sentence-level BLEU.
It is recommended to enable `effective_order` for sentence-level BLEU.
It is recommended to enable `effective_order` for sentence-level BLEU.
It is recommended to enable `effective_order` for sentence-level BLEU.
It is recommended to enable `effective_order` for sentence-level BLEU.


[0.00021454281162072068,
 0.04513645009838105,
 6.158879880353981e-05,
 0.00621707886415799,
 0.0031085394320789954,
 0.03141223826058685,
 0.0079914090191687,
 1.5137615765591065e-06,
 0.0020332857927571266,
 2.8511344810546342e-05]

In [71]:
print(statistics.mean(bleu_scores))
print(statistics.variance(bleu_scores))

0.009620515818394208
0.00024609413190231435


In [72]:
rouge_scorer = Rouge()

rouge_scores = [rouge_scorer.get_scores(
    hyps=description,
    refs=reference,
)[0]["rouge-l"]["f"] for reference in top_10_recommendations[column].tolist()]
# score[0]["rouge-l"]["f"]

rouge_scores

[0.03999999773800013,
 0.0563380251775443,
 0.05555555391300159,
 0.08695651931238192,
 0.09523809306848077,
 0.07228915398461325,
 0.059999997738000084,
 0.03488371953285565,
 0.09708737643510233,
 0.0714285697438776]

In [73]:
print(statistics.mean(rouge_scores))
print(statistics.variance(rouge_scores))

0.06697770066438576
0.00046695487437955867
